In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch import nn
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
# Dataset from https://www.kaggle.com/datasets/saurabhshahane/electricity-load-forecasting
df = pd.read_csv('kaggle_electricity_load_forecasting_train.csv')
df = df.set_index('datetime')

In [3]:
data_output = np.array(df['nat_demand']).reshape(-1, 1)
data_input = np.array(df[['T2M_toc', 'QV2M_toc','TQL_toc','W2M_toc']])

In [4]:
data_output = torch.Tensor(data_output).cuda()
data_input = torch.Tensor(data_input).cuda()

X_train, X_test, y_train, y_test = train_test_split(data_input, data_output, test_size=0.2)

In [5]:
class AttentionNet(nn.Module):
    def __init__(self, inp_size, out_size, linear_size=36):
        super(AttentionNet, self).__init__()

        self.inp_size = inp_size
        self.linear_size = linear_size
        self.out_size = out_size
        
        self.inp_lin = nn.Linear(self.inp_size, self.linear_size)
        self.lin1 = nn.Linear(self.linear_size, self.linear_size)
        self.lin2 = nn.Linear(self.linear_size, self.linear_size)
        self.out_lin = nn.Linear(self.linear_size, self.out_size)
    def forward(self, x):
        x = self.inp_lin(x)
        x1 = F.relu(self.lin1(x))
        x2 = F.relu(self.lin2(x))
        temp = torch.matmul(x1, torch.matmul(x2.transpose(-2, -1), x2))
        x = self.out_lin(temp)
        return x


class DLFTNet(nn.Module):
    def __init__(self, inp_size, out_size, linear_size=36):
        super(DLFTNet, self).__init__()

        self.inp_size = inp_size
        self.linear_size = linear_size
        self.out_size = out_size
        
        self.inp_lin = nn.Linear(self.inp_size, self.linear_size)
        self.lin1 = nn.Linear(self.linear_size, self.linear_size)
        self.out_lin = nn.Linear(self.linear_size ** 2, self.out_size)
        
    def forward(self, x):
        x = self.inp_lin(x)
        x1 = torch.fft.fft(x).real.view(-1, x.shape[1], 1)
        x2 = F.relu(self.lin1(x).view(-1, 1, x.shape[1]))
        temp = torch.matmul(x1, x2).reshape(-1, x.shape[1] ** 2)
        x = self.out_lin(temp)
        return x

class MLPNet(nn.Module):
    def __init__(self, inp_size, out_size, freq_domain=None, linear_size=36):
        super(MLPNet, self).__init__()

        self.inp_size = inp_size
        self.linear_size = linear_size
        self.out_size = out_size
        self.freq_domain = freq_domain
        
        self.inp_lin = nn.Linear(self.inp_size, self.linear_size)
        self.lin1 = nn.Linear(self.linear_size, self.linear_size)
        self.out_lin = nn.Linear(self.linear_size, self.out_size)
        
    def forward(self, x):
        x = self.inp_lin(x)
        x1 = F.relu(self.lin1(x))
        x = self.out_lin(x1)
        return x

In [6]:
net = MLPNet(4, 1).cuda()
opti = torch.optim.Adam(net.parameters(), lr=0.0001)
loss = torch.nn.L1Loss().cuda()

for i in range(0, 10000) :
    opti.zero_grad()
    out = net(X_train)
    l = loss(out, y_train)
    l.backward()
    opti.step()
    if i % 100 == 0 :
        print(i, l.item())

out = net(X_test).detach().cpu().numpy()
cpu_y_test = y_test.detach().cpu().numpy()
print(root_mean_squared_error(cpu_y_test, out))

C:\Users\flyin\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 1181.97119140625
100 1177.33447265625
200 1171.414794921875
300 1163.2901611328125
400 1151.924560546875
500 1136.311279296875
600 1115.4874267578125
700 1088.7666015625
800 1055.5067138671875
900 1015.1172485351562
1000 967.150146484375
1100 911.2005004882812
1200 846.8927001953125
1300 773.8929443359375
1400 691.8914184570312
1500 600.6033935546875
1600 500.0847473144531
1700 394.53973388671875
1800 299.57269287109375
1900 234.61572265625
2000 202.42921447753906
2100 189.1259307861328
2200 183.89242553710938
2300 181.16310119628906
2400 179.09420776367188
2500 177.16004943847656
2600 175.23635864257812
2700 173.29843139648438
2800 171.34446716308594
2900 169.37982177734375
3000 167.4129638671875
3100 165.45407104492188
3200 163.5090789794922
3300 161.59129333496094
3400 159.7122802734375
3500 157.85797119140625
3600 156.04156494140625
3700 154.26397705078125
3800 152.5464324951172
3900 150.9071044921875
4000 149.33517456054688
4100 147.8204803466797
4200 146.3750457763672
4300 144.

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
net = AttentionNet(4, 1).cuda()
opti = torch.optim.Adam(net.parameters(), lr=0.0001)
loss = torch.nn.L1Loss().cuda()

for i in range(0, 10000) :
    opti.zero_grad()
    out = net(X_train)
    l = loss(out, y_train)
    l.backward()
    opti.step()
    if i % 100 == 0 :
        print(i, l.item())

out = net(X_test).detach().cpu().numpy()
cpu_y_test = y_test.detach().cpu().numpy()
print(root_mean_squared_error(cpu_y_test, out))

In [ ]:
net = DLFTNet(4, 1).cuda()
opti = torch.optim.Adam(net.parameters(), lr=0.0001)
loss = torch.nn.L1Loss().cuda()

for i in range(0, 10000) :
    opti.zero_grad()
    out = net(X_train)
    l = loss(out, y_train)
    l.backward()
    opti.step()
    if i % 100 == 0 :
        print(i, l.item())

out = net(X_test).detach().cpu().numpy()
cpu_y_test = y_test.detach().cpu().numpy()
print(root_mean_squared_error(cpu_y_test, out))